# Implementation Of Proposed Method in 
#"Arbitrary Shape Scene Text Detection with Adaptive Text Region Representation"

**Author: Maryam Sadat Hshemi , Sara Aein**

**Download papre : https://arxiv.org/abs/1905.05980**


## 1.Text_RPN

### Import Prerequesties

In [0]:
import keras
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, MaxPool2D, GlobalAveragePooling2D, multiply

### Squeeze and Excitation Block

In [0]:
def SE_Block(in_block, name = '', ratio=16):
    shape = in_block.shape.as_list()
    filters = shape[-1]

    x = GlobalAveragePooling2D(name = name + '_GlobalAvgPool')(in_block)
    x = Dense(filters // ratio, activation='relu',use_bias= False, name = name + '_1')(x)
    x = Dense(filters, activation='sigmoid',use_bias= False, name = name + '_2')(x)
    x = multiply([in_block,x],name = name + '_multiply')
    return x

### SE_VGG16
This is a backbone network of Text_RPN. 

In [0]:
def SE_VGG16(img_shape):
  input = Input(img_shape, name = 'input')
  conv1_1 = Conv2D(filters = 64, kernel_size =(3,3), padding = 'same', activation ='relu', name = 'conv1_1')(input)
  conv1_2 = Conv2D(filters = 64, kernel_size =(3,3), padding = 'same', activation ='relu', name = 'conv1_2')(conv1_1)
  pool1 = MaxPool2D(pool_size = (2,2), strides = (2,2), name = 'pool1')(conv1_2)
  se1 = SE_Block(pool1,'se1')

  conv2_1 = Conv2D(filters = 128, kernel_size =(3,3), padding = 'same', activation ='relu', name = 'conv2_1')(se1)
  conv2_2 = Conv2D(filters = 128, kernel_size =(3,3), padding = 'same', activation ='relu', name = 'conv2_2')(conv2_1)
  pool2 = MaxPool2D(pool_size = (2,2), strides = (2,2), name = 'pool2')(conv2_2)
  se2 = SE_Block(pool2,'se2')

  conv3_1 = Conv2D(filters = 256, kernel_size =(3,3), padding = 'same', activation ='relu', name = 'conv3_1')(se2)
  conv3_2 = Conv2D(filters = 256, kernel_size =(3,3), padding = 'same', activation ='relu', name = 'conv3_2')(conv3_1)
  conv3_3 = Conv2D(filters = 256, kernel_size =(3,3), padding = 'same', activation ='relu', name = 'conv3_3')(conv3_2)
  pool3 = MaxPool2D(pool_size = (2,2), strides = (2,2), name = 'pool3')(conv3_3)
  se3 = SE_Block(pool3, 'se3')

  conv4_1 = Conv2D(filters = 512, kernel_size =(3,3), padding = 'same', activation ='relu', name = 'conv4_1')(se3)
  conv4_2 = Conv2D(filters = 512, kernel_size =(3,3), padding = 'same', activation ='relu', name = 'conv4_2')(conv4_1)
  conv4_3 = Conv2D(filters = 512, kernel_size =(3,3), padding = 'same', activation ='relu', name = 'conv4_3')(conv4_2)
  pool4 = MaxPool2D(pool_size = (2,2), strides = (2,2), name = 'pool4')(conv4_3)
  se4 = SE_Block(pool4, 'se4')

  conv5_1 = Conv2D(filters = 512, kernel_size =(3,3), padding = 'same', activation ='relu', name = 'conv5_1')(se4)
  conv5_2 = Conv2D(filters = 512, kernel_size =(3,3), padding = 'same', activation ='relu', name = 'conv5_2')(conv5_1)
  conv5_3 = Conv2D(filters = 512, kernel_size =(3,3), padding = 'same', activation ='relu', name = 'conv5_3')(conv5_2)
  pool5 = MaxPool2D(pool_size = (2,2), strides = (2,2), name = 'pool4')(conv4_3)
  se5 = SE_Block(pool5,'se5')

  model = Model(inputs=[input], outputs=[se5]) 
  model.summary()
  return model

### Generate Anchor Boxes

### Region Proposal Network

In [0]:
def RPN(train_path, num_anchor_box_scales, num_anchor_box_ratios):
    # -*- coding: utf-8 -*-
    # stuff
    from __future__ import division
    import random
    import pprint
    import keras
    import sys
    import time
    import numpy as np
    from optparse import OptionParser
    import pickle
    import os

    from keras import backend as K
    from keras.optimizers import Adam, SGD, RMSprop
    from keras.layers import Input
    from keras.models import Model
    from keras_frcnn import data_generators
    from keras_frcnn import config
    from keras_frcnn import losses as losses
    import keras_frcnn.roi_helpers as roi_helpers
    from keras.utils import generic_utils

    # # gpu setting
    # if 'tensorflow' == K.backend():
    #     import tensorflow as tf
    # from keras.backend.tensorflow_backend import set_session
    # config2 = tf.ConfigProto()
    # config2.gpu_options.allow_growth = True
    # set_session(tf.Session(config=config2))

    # make dirs to save rpn
    # "./models/rpn/rpn"
    if not os.path.isdir("models"):
      os.mkdir("models")
    if not os.path.isdir("models/rpn"):
      os.mkdir("models/rpn")


    from keras_frcnn.pascal_voc_parser import get_data

    # pass the settings from the command line, and persist them in the config object
    C = config.Config()

    # set data argumentation
    C.use_horizontal_flips = False
    C.use_vertical_flips = False
    C.rot_90 = False

    C.model_path = './model_frcnn.hdf5'
    C.num_rois = 10

    C.network = 'vgg16'
    from keras_frcnn import vgg as nn

    C.base_net_weights = nn.get_weight_path()


    # place weight files on your directory
    base_net_weights = nn.get_weight_path()


    #### load images here ####
    # get voc images

    # '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    all_imgs, classes_count, class_mapping = get_data(train_path)

    print(classes_count)

    # add background class as 21st class
    if 'bg' not in classes_count:
      classes_count['bg'] = 0
      class_mapping['bg'] = len(class_mapping)

    C.class_mapping = class_mapping

    inv_map = {v: k for k, v in class_mapping.items()}

    print('Training images per class:')
    pprint.pprint(classes_count)
    print('Num classes (including bg) = {}'.format(len(classes_count)))

    config_output_filename = "config.pickle"

    with open(config_output_filename, 'wb') as config_f:
      pickle.dump(C,config_f)
      print('Config has been written to {}, and can be loaded when testing to ensure correct results'.format(config_output_filename))

    random.shuffle(all_imgs)

    num_imgs = len(all_imgs)

    # split to train and val
    train_imgs = [s for s in all_imgs if s['imageset'] == 'trainval']
    val_imgs = [s for s in all_imgs if s['imageset'] == 'test']

    print('Num train samples {}'.format(len(train_imgs)))
    print('Num val samples {}'.format(len(val_imgs)))


    data_gen_train = data_generators.get_anchor_gt(train_imgs, classes_count, C, nn.get_img_output_length, K.image_dim_ordering(), mode='train')
    data_gen_val = data_generators.get_anchor_gt(val_imgs, classes_count, C, nn.get_img_output_length,K.image_dim_ordering(), mode='val')

    # set input shape
    input_shape_img = (None, None, 3)

    img_input = Input(shape=input_shape_img)
    roi_input = Input(shape=(None, 4))

    # create rpn model here
    # define the base network (resnet here, can be VGG, Inception, etc)
    shared_layers = nn.nn_base(img_input, trainable=True)

    # define the RPN, built on the base layers
    # rpn outputs regression and cls
    num_anchors = num_anchor_box_scales * num_anchor_box_ratios
    rpn = nn.rpn(shared_layers, num_anchors)

    model_rpn = Model(img_input, rpn[:2])

    #load weights from pretrain
    try:
      print('loading weights from {}'.format(C.base_net_weights))
      model_rpn.load_weights(C.base_net_weights, by_name=True)
    #	model_classifier.load_weights(C.base_net_weights, by_name=True)
      print("loaded weights!")
    except:
      print('Could not load pretrained model weights. Weights can be found in the keras application folder \
        https://github.com/fchollet/keras/tree/master/keras/applications')

    # compile model
    optimizer = Adam(lr=1e-5, clipnorm=0.001)
    model_rpn.compile(optimizer=optimizer, loss=[losses.rpn_loss_cls(num_anchors), losses.rpn_loss_regr(num_anchors)])
    model_rpn.summary()

    # write training misc here
    epoch_length = 100
    num_epochs = 50
    iter_num = 0

    losses = np.zeros((epoch_length, 5))
    rpn_accuracy_rpn_monitor = []
    rpn_accuracy_for_epoch = []
    start_time = time.time()

    best_loss = np.Inf

    class_mapping_inv = {v: k for k, v in class_mapping.items()}
    print('Starting training')

    vis = True

    # start acutual training here
    #X, Y, img_data = next(data_gen_train)
    #
    ##loss_rpn = model_rpn.train_on_batch(X, Y)
    #P_rpn = model_rpn.predict_on_batch(X)

    # you should enable NMS when you visualize your results.
    # NMS will filter out redundant predictions rpn gives, and will only leave the "best" predictions.
    # P_rpn = model_rpn.predict_on_batch(image)
    # R = roi_helpers.rpn_to_roi(P_rpn[0], P_rpn[1], C, K.image_dim_ordering(), use_regr=True, overlap_thresh=0.7, max_boxes=300)
    # X2, Y1, Y2, IouS = roi_helpers.calc_iou(R, img_data, C, class_mapping)
    # this will output the binding box axis. [x1,x2,y1,y2].

    Callbacks=keras.callbacks.ModelCheckpoint("./models/rpn/rpn."+C.network+".weights.{epoch:02d}-{loss:.2f}.hdf5", monitor='loss', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=4)
    callback=[Callbacks]
    if len(val_imgs) == 0:
        # assuming you don't have validation data
        history = model_rpn.fit_generator(data_gen_train,
                        epochs=num_epochs, steps_per_epoch = 1000, callbacks=callback)
        loss_history = history.history["loss"]
    else:
        history = model_rpn.fit_generator(data_gen_train,
                        epochs=num_epochs, validation_data=data_gen_val,
                        steps_per_epoch=1000, callbacks=callback, validation_steps=100)
        loss_history = history.history["val_loss"]

    import numpy
    numpy_loss_history = numpy.array(loss_history)
    numpy.savetxt(C.network+"_rpn_loss_history.txt", numpy_loss_history, delimiter=",")

## 2.Text Refinement Network